### WordEmbsAug using Word2vector
Example: Original Sentence: move green block to the left. Augmented Sentence: ['move quasi_governmental green to the McCuin_dialed_authorities']

### ContextualWordEmbsAug(model_path='bert-base-uncased', substitute)
Example: Original Sentence: pick the pink block. Augmented Sentence: ['organizational name pink block']

### SynonymAug(aug_src='wordnet')
Example: Original Sentence: pick the pink block. Augmented Sentence: ['blame the pink block']. Words can be customized. However, the entire scentence cannot be controlled in the other contextual techniques.

# Original Dataset

In [91]:
import pandas as pd
df1 = pd.read_csv("OriginalHumanGenMRE.csv")
df1
# removing rwos that modality column values is nan
df1 = df1.dropna(subset=[" HumanUsedModality"])
df1

,Date,PromptStartTime,UserSpeech,HumanUsedModality,FocusObj,FocusPosition,TargetObj,TargetObjPosition,Demonstrative,Relations,Configurations,EventsHistory,ObjectsHistory
0,2023/10/29,15:10:28,No speech,pointing only,YellowBlock1,(-0.033886610: 1.124870000: -0.297074900),None,( -0.2326699: 1.12487: -0.4550126) ),None,"in_front(YellowBlock2,BlueBlock1) and under(Fl...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
1,2023/10/29,15:10:28,No speech,pointing only,YellowBlock1,(-0.033886610: 1.124870000: -0.297074900),None,( -0.2326699: 1.12487: -0.4550126) ),None,"hold(Diana2,YellowBlock1) and in_front(YellowB...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
2,2023/10/29,15:10:37,No speech,pointing only,YellowBlock1,(-0.232669900: 1.124870000: -0.455012600),GreenBlock2,(0.059132730: 1.124870000: 0.140608100),None,"in_front(YellowBlock2,BlueBlock1) and under(Fl...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
3,2023/10/29,15:10:37,No speech,pointing only,YellowBlock1,(-0.232669900: 1.124870000: -0.455012600),GreenBlock2,(0.059132730: 1.124870000: 0.140608100),None,"in_front(YellowBlock2,BlueBlock1) and hold(Dia...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
4,2023/10/29,15:10:41,No speech,pointing only,RedBlock2,(0.283733800: 1.124870000: 0.384224100),None,( -0.2955508: 1.12487: 0.3969399) ),None,"in_front(YellowBlock2,BlueBlock1) and touching...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put put,RedBlock2 YellowBlock1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10403,12/22/23,13:28:29,put it on the red block next to the green block,speech only,RedBlock1,(-0.202601500: 1.124870000: 0.328684200),RedBlock2,(-0.503365200: 1.724870000: 0.263937400),the,"under(Floor,YellowBlock2) and under(Floor,Pink...","RedBlock1/ (-0.202601500,1.124870000,0.3286842...",put,RedBlock1
10404,12/22/23,13:36:56,pick up the pink block,speech only,PinkBlock1,(-0.302593200: 1.124870000: 0.209513600),None,None,the,"right & touching(RedBlock1,plate) and in_front...","RedBlock1/ (0.117900100,1.125058000,0.02637539...",grasp,PinkBlock1
10405,12/22/23,13:37:31,put it next to the plate,speech only,PinkBlock1,(0.472000500: 1.124774000: 0.026368300),None,None,the,"touching(YellowBlock2,Table) and right & touch...","RedBlock1/ (0.117900100,1.125058000,0.02637539...",put,PinkBlock1
10406,12/22/23,14:01:42,put it on the green block next to the plate,speech only,GreenBlock1,(0.246942200: 1.124870000: -0.266895800),GreenBlock2,(-0.015179130: 1.124870000: -0.280726300),the,"under(Floor,GreenBlock2) and under(Table,Yello...","RedBlock1/ (-0.449597000,1.124870000,0.2213885...",put,GreenBlock1


In [58]:
# input: FocusOb, FocusPosition,TargetObj,TargetObjPosition, Relations, Configurations, Previous events
# output: UserSpeech, Demonstrative, HumanUsedModality, FocusPosition

original_dataset = [
    {
        "instruction": "Generate a multimodal expression for an object.",
        "input": str(row_dict1[" FocusObj"]) + " , " + str(row_dict1[" FocusPosition"]) + " , " + str(row_dict1[" TargetObj"])  + " , " + str(row_dict1["TargetObjPosition"]) + " , " + str(row_dict1[" Relations"])+ " , " + str(row_dict1["Configurations"])+ " , " + str(row_dict1[" EventsHistory"])+ " , " + str(row_dict1[" ObjectsHistory"]),
        "output":str(row_dict1[" UserSpeech"])+ " , "+ str(row_dict1[" Demonstrative"])+ " , "+ str(row_dict1[" HumanUsedModality"])+ " , "+ str(row_dict1[" FocusPosition"])
    }
    for row_dict1 in df1.to_dict(orient="records")
]
#original_dataset[0]

In [59]:
import json
with open("OriginalHumanGenMRE.json", "w") as f:
    json.dump(original_dataset, f)

# Augmented Dataset

In [60]:
res=[]
orignial_date = []
orignial_startTime = []
orignial_fo = []
orignial_fp = []
orignial_to = []
orignial_tp = []
orignial_dem = []
orignial_rel = []
orignial_tp = []
orignial_con = []
orignial_eh = []
orignial_oh = []
orignial_mod= []
count3=0
for i in df1.index:

    re=df1[' UserSpeech'].loc[i]
    date=df1['Date'].loc[i]
    startTime=df1[' PromptStartTime'].loc[i]
    fo=df1[' FocusObj'].loc[i]
    fp=df1[' FocusPosition'].loc[i]
    to=df1[' TargetObj'].loc[i]
    tp=df1['TargetObjPosition'].loc[i]
    dem=df1[' Demonstrative'].loc[i]
    rel=df1[' Relations'].loc[i]
    con=df1['Configurations'].loc[i]
    eh=df1[' EventsHistory'].loc[i]
    oh=df1[' ObjectsHistory'].loc[i]
    mod=df1[' HumanUsedModality'].loc[i]
    if (re != "No speech" and re != "nan" and re != "nevermind" and re != "no" and re != "yes" and re != "stop"):

            
#         if ("." in re):
#             re=re.replace("."," ")
            if "(" in str(re):
                re=str(re).replace("("," ") 
                re=str(re).replace(")"," ") 
            if "." in str(re):
                re=str(re).replace("."," ") 
            if "\\" in str(re): 
                re=str(re).replace("\\"," ") 
            if "blocks" in str(re): 
                re=str(re).replace("blocks","block") 
            if "it" in str(re): 
                re=str(re).replace("it","this") 
            if "plates" in str(re): 
                re=str(re).replace("plates","plate") 
            if "block'" in str(re): 
                re=str(re).replace("block'","block") 
            if "place'" in str(re): 
                re=str(re).replace("place","put") 
            if "place'" in str(re): 
                re=str(re).replace("whthise","white") 
            if "blog'" in str(re): 
                re=str(re).replace("blog","block") 
            if "come on'" in str(re): 
                re=str(re).replace("come on","") 
            if "let's focus on" in str(re): 
                re=str(re).replace("let's focus on","") 
            if "yeah" in str(re): 
                re=str(re).replace("yeah","") 
            if "let's" in str(re): 
                re=str(re).replace("let's","") 
            if "lift" in str(re): 
                re=str(re).replace("lift","pick up") 
            
            
            res.append(re) 
            orignial_date.append(date)
            orignial_startTime.append(startTime)
            orignial_fo.append(fo)
            orignial_fp.append(fp)
            orignial_to.append(to)
            orignial_tp.append(tp)
            orignial_dem.append(dem)
            orignial_rel.append(rel)
            orignial_con.append(con)
            orignial_eh.append(eh)
            orignial_oh.append(oh)
            orignial_mod.append(mod)



# for a in res:
#     count3=count3+1
#     print(count3, "re: ",a) 


In [61]:
#Data augmentation 

import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import operator
# from textattack.augmentation import EasyDataAugmenter

# def DataAug(sentences):

# Original sentences
original_sentences = []
original_sentences2=[]
original_sentences= res

# features for augmented scentences
aug_date = []
aug_startTime = []
aug_fo = []
aug_fp = []
aug_to = []
aug_tp = []
aug_dem = []
aug_rel = []
aug_tp = []
aug_con = []
aug_eh = []
aug_oh = []
aug_mod = []


# Words to protect in each sentence
words_to_protect = ["block", "red", "green", "blue", "yellow", "pink","plate", "cup", "block"," here "]

# Placeholder for protected words
placeholder = "entity"

# Augmenter initialization
aug = naw.SynonymAug(aug_src='wordnet',aug_max = 3)
#     contextual_augmenter = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")
#     aug = EasyDataAugmenter()
#     delete_augmenter = naw.RandomWordAug(action="delete")
#     aug = naw.WordEmbsAug(model_type='word2vec', model_path = 'GoogleNews-vectors-negative300.bin',action="substitute")


# Augment each sentence while avoiding the protected words
augmented_sentences=[]
words_to_protect2=[]
sentences = []
dictionary={}
count1=0
count2=0
augmented_sentences.clear()
original_sentences2.clear()

for sentence in original_sentences:    
# Identify and protect specific words in the sentence
    protected_sentence = sentence
    for words_to_protect_in_sentence in words_to_protect:

        if words_to_protect_in_sentence in str(protected_sentence):

                sentences=protected_sentence.split(" ")
                dictionary[sentences.index(words_to_protect_in_sentence)]= words_to_protect_in_sentence

        protected_sentence = str(protected_sentence).replace(words_to_protect_in_sentence, placeholder,1)
        myKeys = list(dictionary.keys())
        myKeys.sort()
        sorted_dict = {i: dictionary[i] for i in myKeys}
        # augment each sentence 3 times 
    augmented_sentence0 = aug.augment(protected_sentence,n=3)


# Replace the placeholder with the original words
    for augsent in augmented_sentence0:
        augmented_sentence=augsent
        for protectWordIndex in sorted_dict:
            augmented_sentence = augmented_sentence.replace(placeholder, sorted_dict[protectWordIndex],1)

        # Replace unusual words with correct ones
        
        if "elevator" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("elevator","pick up") 
        if "annul" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("annul","pick up")    
        if "beak" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("beak","pick")   
        if "clean up" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("clean up","select")  
        if "piece up" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("piece up","select")  
        if "clean" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("clean","select") 
        if "foot up" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("foot up","select") 
        if "nibble" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("nibble","choose") 
        if "seize" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("seize","hold")  
        if "snaffle" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("snaffle","hold") 
        if "lift" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("lift","raise") 
        if "face" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("face","") 
        if "break up" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("break up","lift")
        if "foot" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("foot","lift") 
        if "plunk up" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("plunk up","lift")
        if "plunk" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("plunk","lift")
        if "fault" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("fault"," ")
        if "blame up" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("blame up","choose")
        if "blame" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("blame","choose")
        if "motility" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("motility","slide")
        if "nonpareil" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("nonpareil","object")
        if "unexpended" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("unexpended","")
        if "astir" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("astir","")
        if "learn" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("learn","grab")
        if "1" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("1","object")
        if "driblet" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("driblet","get rid of")                
        if "unity" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("unity","block")                 
        if "direct" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("direct","hold")
        if "order" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("order","place") 
        if "plectrum" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("plectrum","hold")
        if "improving" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("improving","")
        if "arrange" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("arrange","position") 
        if "plectron" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("plectron","lift") 
        if "hardening" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("hardening","put") 
        if "cloth" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("cloth","") 
        if " ace " in augmented_sentence:
            augmented_sentence=augmented_sentence.replace(" ace ","block")
        if " i " in augmented_sentence:
            augmented_sentence=augmented_sentence.replace(" i ","target")
        if "weft" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("weft","pick")
        if "compass" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("compass","catch") 
        if "woof" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("woof","pick")
        if "cream" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("cream","lift")
        if "put option" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("put option","set")                
        if "option" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("option","pick")
        if "pickax" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("pickax","raise")
        if "former" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("former","second")                
        if "office" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("office","set")   
        if "hither" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("hither","here")                 
        if "cull" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("cull","select")   
        if "frame" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("frame","set")
        if "invest" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("invest","place")  
        if "couch" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("couch","lay") 
        if "strawman" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("strawman","front") 
        if "man" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("man","") 
        if "figurepart" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("figurepart","front")  
        if "forehead" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("forehead","front")                 
        if "commit" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("place","lay")                 
        if "redact" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("redact","position")                 
        if "remaining" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("remaining","left")                 
        if "helping" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("helping","")
        if "seat" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("seat","")
        if "unrivaled" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("unrivaled","object")                
        if "choice" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("choice","choose")
        if "picking" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("picking","choose")                 
        if "selection" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("selection","select") 
        if "filling" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("filling","lift") 
        if "movement" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("movement","slide")  
        if "whirligig" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("whirligig","topmost") 
        if "incite" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("incite","transit")
        if "piazza" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("piazza","place")
        if "prompt" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("prompt","place")
        if "go" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("go","transfer")
        if "impress" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("impress","slide")
        if "travel" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("travel","transit")
        if "peerless" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("peerless","object")                
        if "cast" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("cast","set")
        if "assume" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("assume","hold")  
        if "lease" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("lease","hold")
        if "involve" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("involve","grab")
        if "read" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("read","grasp")
        if "inning of" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("inning of","")
        if "strike" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("strike","transit")
        if "affect" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("affect","slide")
        if "act" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("act","set")
        if "Yellowness" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("Yellowness","Yellow")
        if "motivate" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("motivate","transfer")
        if "be active" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("be active","slide")
        if "snatch" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("snatch","lift")
        if "early unripe" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("early unripe","")
        if "leave alone" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("leave alone","left")
        if "leave" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("leave","left")
        if "auction" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("auction","")
        if "vex" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("vex","have")
        if "receive" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("receive","have")
        if "degree" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("degree","point")  
        if "sire" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("sire","have")
        if "essay" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("essay","have")
        if "presence" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("presence","front")  
        if "recess" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("recess","corner")                                 
        if "unrivaled" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("unrivaled","block")
        if "recession" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("recession","corner")
        if "battlefront" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("battlefront","front")  
        if "unmatchable" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("unmatchable","object")                 
        if "engine" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("engine","")
        if "readiness" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("readiness","place")  
        if "future" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("future","")                   
        if "pluck up" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("pluck up","raise")                   
        if "ski tow" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("ski tow","raise") 
        if "pluck" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("pluck","raise")  
        if "elevation" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("elevation","upper")
        if "peck" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("peck","select") 
        if "bead" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("bead","get rid of") 
        if "unmatched" in augmented_sentence:
            augmented_sentence=augmented_sentence.replace("unmatched","object") 
    
        augmented_sentences.append(augmented_sentence)
#         print(count1+1,sentence)
        original_sentences2.append(str(sentence))
        
        aug_date.append(orignial_date[count1])
        aug_startTime.append(orignial_startTime[count1])
        aug_fo.append(orignial_fo[count1])
        aug_fp.append(orignial_fp[count1])
        aug_to.append(orignial_to[count1])
        aug_tp.append(orignial_tp[count1])
        aug_dem.append(orignial_dem[count1])
        aug_rel.append(orignial_rel[count1])
        aug_con.append(orignial_con[count1])
        aug_eh.append(orignial_eh[count1])
        aug_oh.append(orignial_oh[count1])
        aug_mod.append(orignial_mod[count1])

    words_to_protect2.clear()
    dictionary.clear()
    count1=count1 +1    
    
# for re,date,start,fo,fp,to,tp,dem,rel,con,eh,oh in zip(augmented_sentences,aug_date,aug_startTime,aug_fo,aug_fp,aug_to,aug_tp,aug_dem,aug_rel,aug_con,aug_eh,aug_oh):        
#     print(count2," RE:", aug0, " date: ", date," start",start," fo", fo," fp: ",fp,"to:", to," tp: ",tp, " dem: ",dem, " rel: ",rel, " con: ", con, " eh: ",eh, " oh: ", oh )
#     count2=count2 +1
#     print()


2024-04-04 05:00:44.726091: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/usr/local/anaconda3/2022.08/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


# Evaluation of Speech Augmentation 

In [ ]:
from datasets import load_metric

bertscore_metric = load_metric('bertscore')
bert_scores = bertscore_metric.compute(predictions=augmented_sentences, references=original_sentences2, lang="en")

total1=0
for pre in bert_scores['precision']:
    total1= total1 + pre
average1=total1/6123
print()
print("Bert_precision average: ", average1)

total1=0
average1=0
for re in bert_scores['recall']:
    total1= total1 + re
average1=total1/6123
print()
print("Bert_recall average: ", average1)

total1=0
average1=0
for f1 in bert_scores['f1']:
    total1= total1 + f1
average1=total1/6123
print()
print("Bert_f1 average: ", average1)
print()
print("----------------------------------------------------------------------")
print("bert_scores: ", bert_scores)



In [62]:
# Zip the two lists into a list of tuples
augmented_res_features = list(zip(aug_date,aug_startTime,augmented_sentences,aug_mod,aug_fo,aug_fp,aug_to,aug_tp,
                                  aug_dem,aug_rel,aug_con,aug_eh,aug_oh))


# print(augmented_res_features)

In [63]:
# Convert the list of tuples to a Pandas Dataframe.
df_augmented_res_features = pd.DataFrame(
augmented_res_features, columns = ['Date', ' PromptStartTime',' UserSpeech',' HumanUsedModality', ' FocusObj', ' FocusPosition',
                                  ' TargetObj','TargetObjPosition', ' Demonstrative',' Relations','Configurations',
                                  ' EventsHistory',' ObjectsHistory'])
df_augmented_res_features


,Date,PromptStartTime,UserSpeech,HumanUsedModality,FocusObj,FocusPosition,TargetObj,TargetObjPosition,Demonstrative,Relations,Configurations,EventsHistory,ObjectsHistory
0,2023/10/29,15:57:16,take hold of the pink block,multimodal,PinkBlock2,(0.155394000: 1.224870000: 0.313696100),None,None,the,"behind(plate,YellowBlock2) and right(cup,RedBl...","RedBlock1(0.155393600,1.124870000,0.313696100)...",put put,GreenBlock1 PinkBlock2
1,2023/10/29,15:57:16,take hold of the pink block,multimodal,PinkBlock2,(0.155394000: 1.224870000: 0.313696100),None,None,the,"behind(plate,YellowBlock2) and right(cup,RedBl...","RedBlock1(0.155393600,1.124870000,0.313696100)...",put put,GreenBlock1 PinkBlock2
2,2023/10/29,15:57:16,grab the pink block,multimodal,PinkBlock2,(0.155394000: 1.224870000: 0.313696100),None,None,the,"behind(plate,YellowBlock2) and right(cup,RedBl...","RedBlock1(0.155393600,1.124870000,0.313696100)...",put put,GreenBlock1 PinkBlock2
3,2023/10/29,15:57:29,pick upward this,speech only,BlueBlock2,(-0.265158200: 1.124870000: 0.177218200),None,None,the,"behind(plate,YellowBlock2) and right(cup,RedBl...","RedBlock1(0.155393600,1.124870000,0.313696100)...",grasp put put,BlueBlock2 GreenBlock1 PinkBlock2
4,2023/10/29,15:57:29,pick upwardly this,speech only,BlueBlock2,(-0.265158200: 1.124870000: 0.177218200),None,None,the,"behind(plate,YellowBlock2) and right(cup,RedBl...","RedBlock1(0.155393600,1.124870000,0.313696100)...",grasp put put,BlueBlock2 GreenBlock1 PinkBlock2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6115,12/22/23,14:01:42,put this on the green block following to the p...,speech only,GreenBlock1,(0.246942200: 1.124870000: -0.266895800),GreenBlock2,(-0.015179130: 1.124870000: -0.280726300),the,"under(Floor,GreenBlock2) and under(Table,Yello...","RedBlock1/ (-0.449597000,1.124870000,0.2213885...",put,GreenBlock1
6116,12/22/23,14:01:42,put this on the green block next to the plate,speech only,GreenBlock1,(0.246942200: 1.124870000: -0.266895800),GreenBlock2,(-0.015179130: 1.124870000: -0.280726300),the,"under(Floor,GreenBlock2) and under(Table,Yello...","RedBlock1/ (-0.449597000,1.124870000,0.2213885...",put,GreenBlock1
6117,12/22/23,14:02:22,chooseward the yellow block,speech only,YellowBlock1,(-0.348228300: 1.124870000: -0.343382100),None,None,the,"under(Floor,GreenBlock2) and under(Table,Yello...","RedBlock1/ (-0.449597000,1.124870000,0.2213885...",grasp,YellowBlock1
6118,12/22/23,14:02:22,pick up the yellow block,speech only,YellowBlock1,(-0.348228300: 1.124870000: -0.343382100),None,None,the,"under(Floor,GreenBlock2) and under(Table,Yello...","RedBlock1/ (-0.449597000,1.124870000,0.2213885...",grasp,YellowBlock1


# Original with Augmented Data

In [64]:
# Vertically concat the original REs with the augmented REs
# original_df_augmented = pd.concat([df1, df_augmented_res_features], axis = 0)
origin_df_augmented = pd.concat([df1, df_augmented_res_features], ignore_index=True)
# print(origin_df_augmented)
origin_df_augmented

,Date,PromptStartTime,UserSpeech,HumanUsedModality,FocusObj,FocusPosition,TargetObj,TargetObjPosition,Demonstrative,Relations,Configurations,EventsHistory,ObjectsHistory
0,2023/10/29,15:10:28,No speech,pointing only,YellowBlock1,(-0.033886610: 1.124870000: -0.297074900),None,( -0.2326699: 1.12487: -0.4550126) ),None,"in_front(YellowBlock2,BlueBlock1) and under(Fl...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
1,2023/10/29,15:10:28,No speech,pointing only,YellowBlock1,(-0.033886610: 1.124870000: -0.297074900),None,( -0.2326699: 1.12487: -0.4550126) ),None,"hold(Diana2,YellowBlock1) and in_front(YellowB...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
2,2023/10/29,15:10:37,No speech,pointing only,YellowBlock1,(-0.232669900: 1.124870000: -0.455012600),GreenBlock2,(0.059132730: 1.124870000: 0.140608100),None,"in_front(YellowBlock2,BlueBlock1) and under(Fl...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
3,2023/10/29,15:10:37,No speech,pointing only,YellowBlock1,(-0.232669900: 1.124870000: -0.455012600),GreenBlock2,(0.059132730: 1.124870000: 0.140608100),None,"in_front(YellowBlock2,BlueBlock1) and hold(Dia...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
4,2023/10/29,15:10:41,No speech,pointing only,RedBlock2,(0.283733800: 1.124870000: 0.384224100),None,( -0.2955508: 1.12487: 0.3969399) ),None,"in_front(YellowBlock2,BlueBlock1) and touching...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put put,RedBlock2 YellowBlock1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16522,12/22/23,14:01:42,put this on the green block following to the p...,speech only,GreenBlock1,(0.246942200: 1.124870000: -0.266895800),GreenBlock2,(-0.015179130: 1.124870000: -0.280726300),the,"under(Floor,GreenBlock2) and under(Table,Yello...","RedBlock1/ (-0.449597000,1.124870000,0.2213885...",put,GreenBlock1
16523,12/22/23,14:01:42,put this on the green block next to the plate,speech only,GreenBlock1,(0.246942200: 1.124870000: -0.266895800),GreenBlock2,(-0.015179130: 1.124870000: -0.280726300),the,"under(Floor,GreenBlock2) and under(Table,Yello...","RedBlock1/ (-0.449597000,1.124870000,0.2213885...",put,GreenBlock1
16524,12/22/23,14:02:22,chooseward the yellow block,speech only,YellowBlock1,(-0.348228300: 1.124870000: -0.343382100),None,None,the,"under(Floor,GreenBlock2) and under(Table,Yello...","RedBlock1/ (-0.449597000,1.124870000,0.2213885...",grasp,YellowBlock1
16525,12/22/23,14:02:22,pick up the yellow block,speech only,YellowBlock1,(-0.348228300: 1.124870000: -0.343382100),None,None,the,"under(Floor,GreenBlock2) and under(Table,Yello...","RedBlock1/ (-0.449597000,1.124870000,0.2213885...",grasp,YellowBlock1


In [ ]:
# input: FocusOb, FocusPosition,TargetObj,TargetObjPosition, Relations, Configurations
# output: UserSpeech, Demonstrative, HumanUsedModality, FocusPosition

AugDataset = [
    {
        "instruction": "Generate a multimodal expression for an object.",
        "input": str(row_dict1[" FocusObj"]) + " , " + str(row_dict1[" FocusPosition"]) + " , " + str(row_dict1[" TargetObj"])  + " , " + str(row_dict1["TargetObjPosition"]) + " , " + str(row_dict1[" Relations"])+ " , " + str(row_dict1["Configurations"])+ " , " + str(row_dict1[" EventsHistory"])+ " , " + str(row_dict1[" ObjectsHistory"]),
        "output":str(row_dict1[" UserSpeech"])+ " , "+ str(row_dict1[" Demonstrative"])+ " , "+ str(row_dict1[" HumanUsedModality"])+ " , "+ str(row_dict1[" FocusPosition"])
    }
    for row_dict1 in origin_df_augmented.to_dict(orient="records")
]
AugDataset[0]

In [ ]:
import json
with open("Aug_HumanGenMRE.json", "w") as f:
   json.dump(AugDataset, f)
AugDataset[0]

In [ ]:
from datasets import load_dataset

data = load_dataset("json", data_files="Aug_HumanGenMRE.json")
data["train"][0]

In [ ]:
train_val = data["train"].train_test_split(test_size=0.2, shuffle=True, seed=42)
val_test = train_val["test"].train_test_split(test_size=0.2, shuffle=True, seed=42)

train_data = (train_val["train"])
val_data = (val_test["train"])
test_data = (val_test["test"])

train_data,val_data,test_data[3]

# Synthesizing Augmented Data

In [ ]:
synth_res=[]

date = []
startTime = []
speechs =[]
fo = []
fp = []
to = []
tp = []
dem = []
rel = []
tp = []
con = []
eh = []
oh = []
mod = []


Count = 0
df_synthREs=None

for i in origin_df_augmented.index:
    
    modality=origin_df_augmented[' HumanUsedModality'].loc[i]
    speech=origin_df_augmented[' UserSpeech'].loc[i]

    if("speech only" in str(modality)):
        focusObj=origin_df_augmented[' FocusObj'].loc[i]
        focusPos=origin_df_augmented[' FocusPosition'].loc[i]
        for j in origin_df_augmented.index:
            modality1=origin_df_augmented[' HumanUsedModality'].loc[j]
            if("pointing only" in str(modality1)):
                focusObj1=origin_df_augmented[' FocusObj'].loc[j]
                focusPos1=origin_df_augmented[' FocusPosition'].loc[j]  
                if (focusObj == focusObj1 and focusPos == focusPos1): 
                    Count=Count+1

                    sample0 = origin_df_augmented.loc[i]   
                    sample1 = sample0
                    sample1[' HumanUsedModality'] = sample1[' HumanUsedModality'].replace('speech only', 'multimodal')

                    sample2=sample0
                    sample2[' HumanUsedModality'] = sample2[' HumanUsedModality'].replace('pointing only', 'multimodal')
                    sample2[' UserSpeech'] = sample2[' UserSpeech'].replace('No speech', speech)

                    print("Count:",Count )
                    print("----------------------------------------------------------------")
                    print("----------------------------------------------------------------")
                    print("speech modification:")
                    print("----------------------------------------------------------------")
                    print(sample1)

                    date.append(sample1[0])
                    startTime.append(sample1[1])
                    speechs.append(sample1[2])
                    mod.append(sample1[3])
                    fo.append(sample1[4])
                    fp.append(sample1[5])
                    to.append(sample1[6])
                    tp.append(sample1[7])
                    dem.append(sample1[8])
                    rel.append(sample1[9])
                    con.append(sample1[10])
                    eh.append(sample1[11])
                    oh.append(sample1[12])

#                      synth_res.append(sample1)
                    print("pointing only modification:")
                    print("----------------------------------------------------------------")
                    print(sample2)
                    date.append(sample1[0])
                    startTime.append(sample2[1])
                    speechs.append(sample2[2])
                    mod.append(sample2[3])
                    fo.append(sample2[4])
                    fp.append(sample2[5])
                    to.append(sample2[6])
                    tp.append(sample2[7])
                    dem.append(sample2[8])
                    rel.append(sample2[9])
                    con.append(sample2[10])
                    eh.append(sample2[11])
                    oh.append(sample2[12])

#                         temp_df_synthREs = pd.concat([sample1, sample2], ignore_index=True)
#                         df_synthREs = pd.concat([temp_df_synthREs,df_synthREs], ignore_index=True)


# Concatenating Synthesized and Augmented Data

In [82]:
synth_res_features = list(zip(date,startTime,speechs,mod,fo,fp,to,tp,dem,rel,con,eh,oh))
df_synth_res_features = pd.DataFrame(synth_res_features, columns = ['Date', ' PromptStartTime',' UserSpeech',' HumanUsedModality', ' FocusObj', ' FocusPosition',
' TargetObj','TargetObjPosition', ' Demonstrative',' Relations','Configurations',' EventsHistory',' ObjectsHistory'])

df_synth_res_features

,Date,PromptStartTime,UserSpeech,HumanUsedModality,FocusObj,FocusPosition,TargetObj,TargetObjPosition,Demonstrative,Relations,Configurations,EventsHistory,ObjectsHistory
0,2023/11/04,13:47:14,pick up the green block,multimodal,GreenBlock2,(0.265834500: 1.124870000: -0.230943100),None,None,the,"touching(cup,Table) and right(RedBlock2,GreenB...","RedBlock1(0.106797000,1.124870000,-0.440875200...",grasp put,GreenBlock2 YellowBlock2
1,2023/11/04,13:47:14,pick up the green block,multimodal,GreenBlock2,(0.265834500: 1.124870000: -0.230943100),None,None,the,"touching(cup,Table) and right(RedBlock2,GreenB...","RedBlock1(0.106797000,1.124870000,-0.440875200...",grasp put,GreenBlock2 YellowBlock2
2,2023/11/04,13:47:14,pick up the green block,multimodal,GreenBlock2,(0.265834500: 1.124870000: -0.230943100),None,None,the,"touching(cup,Table) and right(RedBlock2,GreenB...","RedBlock1(0.106797000,1.124870000,-0.440875200...",grasp put,GreenBlock2 YellowBlock2
3,2023/11/04,13:47:14,pick up the green block,multimodal,GreenBlock2,(0.265834500: 1.124870000: -0.230943100),None,None,the,"touching(cup,Table) and right(RedBlock2,GreenB...","RedBlock1(0.106797000,1.124870000,-0.440875200...",grasp put,GreenBlock2 YellowBlock2
4,2023/11/04,14:05:27,green block,multimodal,GreenBlock1,(0.349752100: 1.124870000: 0.334546600),None,None,the,"behind & touching(GreenBlock1,plate) and under...","RedBlock1(-0.075182480,1.124870000,-0.21375860...",grasp put put put put,GreenBlock1 GreenBlock1 YellowBlock2 RedBlock2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5627,04/11/1023,14:05:12,choose upward the yellow block in front of you,multimodal,YellowBlock2,(-0.253141600: 1.124870000: -0.219346700),None,None,the,"under(Floor,GreenBlock1) and under(Floor,PinkB...","RedBlock1/ (-0.075182480,1.124870000,-0.213758...",grasp,YellowBlock2
5628,04/11/1023,14:05:12,raise the yellow block in front of you,multimodal,YellowBlock2,(-0.253141600: 1.124870000: -0.219346700),None,None,the,"under(Floor,GreenBlock1) and under(Floor,PinkB...","RedBlock1/ (-0.075182480,1.124870000,-0.213758...",grasp,YellowBlock2
5629,04/11/1023,14:05:12,raise the yellow block in front of you,multimodal,YellowBlock2,(-0.253141600: 1.124870000: -0.219346700),None,None,the,"under(Floor,GreenBlock1) and under(Floor,PinkB...","RedBlock1/ (-0.075182480,1.124870000,-0.213758...",grasp,YellowBlock2
5630,04/11/1023,14:05:12,raise the yellow block in front of you,multimodal,YellowBlock2,(-0.253141600: 1.124870000: -0.219346700),None,None,the,"under(Floor,GreenBlock1) and under(Floor,PinkB...","RedBlock1/ (-0.075182480,1.124870000,-0.213758...",grasp,YellowBlock2


In [83]:
synth_df_OrigOug = pd.concat([origin_df_augmented, df_synth_res_features], ignore_index=True)
synth_df_OrigOug

,Date,PromptStartTime,UserSpeech,HumanUsedModality,FocusObj,FocusPosition,TargetObj,TargetObjPosition,Demonstrative,Relations,Configurations,EventsHistory,ObjectsHistory
0,2023/10/29,15:10:28,No speech,pointing only,YellowBlock1,(-0.033886610: 1.124870000: -0.297074900),None,( -0.2326699: 1.12487: -0.4550126) ),None,"in_front(YellowBlock2,BlueBlock1) and under(Fl...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
1,2023/10/29,15:10:28,No speech,pointing only,YellowBlock1,(-0.033886610: 1.124870000: -0.297074900),None,( -0.2326699: 1.12487: -0.4550126) ),None,"hold(Diana2,YellowBlock1) and in_front(YellowB...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
2,2023/10/29,15:10:37,No speech,pointing only,YellowBlock1,(-0.232669900: 1.124870000: -0.455012600),GreenBlock2,(0.059132730: 1.124870000: 0.140608100),None,"in_front(YellowBlock2,BlueBlock1) and under(Fl...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
3,2023/10/29,15:10:37,No speech,pointing only,YellowBlock1,(-0.232669900: 1.124870000: -0.455012600),GreenBlock2,(0.059132730: 1.124870000: 0.140608100),None,"in_front(YellowBlock2,BlueBlock1) and hold(Dia...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put,YellowBlock1
4,2023/10/29,15:10:41,No speech,pointing only,RedBlock2,(0.283733800: 1.124870000: 0.384224100),None,( -0.2955508: 1.12487: 0.3969399) ),None,"in_front(YellowBlock2,BlueBlock1) and touching...","RedBlock1(0.580661800,1.124870000,-0.239872300...",put put,RedBlock2 YellowBlock1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22154,04/11/1023,14:05:12,choose upward the yellow block in front of you,multimodal,YellowBlock2,(-0.253141600: 1.124870000: -0.219346700),None,None,the,"under(Floor,GreenBlock1) and under(Floor,PinkB...","RedBlock1/ (-0.075182480,1.124870000,-0.213758...",grasp,YellowBlock2
22155,04/11/1023,14:05:12,raise the yellow block in front of you,multimodal,YellowBlock2,(-0.253141600: 1.124870000: -0.219346700),None,None,the,"under(Floor,GreenBlock1) and under(Floor,PinkB...","RedBlock1/ (-0.075182480,1.124870000,-0.213758...",grasp,YellowBlock2
22156,04/11/1023,14:05:12,raise the yellow block in front of you,multimodal,YellowBlock2,(-0.253141600: 1.124870000: -0.219346700),None,None,the,"under(Floor,GreenBlock1) and under(Floor,PinkB...","RedBlock1/ (-0.075182480,1.124870000,-0.213758...",grasp,YellowBlock2
22157,04/11/1023,14:05:12,raise the yellow block in front of you,multimodal,YellowBlock2,(-0.253141600: 1.124870000: -0.219346700),None,None,the,"under(Floor,GreenBlock1) and under(Floor,PinkB...","RedBlock1/ (-0.075182480,1.124870000,-0.213758...",grasp,YellowBlock2


In [ ]:
# input: FocusOb, FocusPosition,TargetObj,TargetObjPosition, Relations, Configurations
# output: UserSpeech, Demonstrative, HumanUsedModality, FocusPosition

SynthDataset = [
    {
        "instruction": "Generate a referring expression for an object.",
        "input": str(row_dict1[" FocusObj"]) + " , " + str(row_dict1[" FocusPosition"]) + " , " + str(row_dict1[" TargetObj"])  + " , " + str(row_dict1["TargetObjPosition"]) + " , " + str(row_dict1[" Relations"])+ " , " + str(row_dict1["Configurations"])+ " , " + str(row_dict1[" EventsHistory"])+ " , " + str(row_dict1[" ObjectsHistory"]),
        "output":str(row_dict1[" UserSpeech"])+ " , "+ str(row_dict1[" Demonstrative"])+ " , "+ str(row_dict1[" HumanUsedModality"])+ " , "+ str(row_dict1[" FocusPosition"])
    }
    for row_dict1 in synth_df_OrigOug.to_dict(orient="records")
]
SynthDataset[0]

In [ ]:
import json
with open("Synth_HumanGenMRE.json", "w") as f:
   json.dump(SynthDataset, f)
SynthDataset[0]

In [ ]:
syndata = load_dataset("json", data_files="Synth_HumanGenMRE.json")
syndata

# Data statistics (original, augmented and synthesized )

In [ ]:
from datasets import load_dataset
# before removing rows that has nan for humanUsedModality column
Augdata0 = load_dataset("json", data_files="HumanGenMRE_RelCon.json")
# after removing rows that has nan for humanUsedModality column
Augdata = load_dataset("json", data_files="Aug_HumanGenMRE.json")

Augdata['train']

In [ ]:
# from datasets import load_dataset
Origdata = load_dataset("json", data_files="OriginalHumanGenMRE.json")
Origdata['train']

In [90]:
aug_mcount=0
aug_pcount=0
aug_scount=0

orig_mcount=0
orig_pcount=0
orig_scount=0

orig_nocount=0
aug_nocount=0

aug_index=0
orig_index=0

for i in Augdata["train"]['output']:
    if ('multimodal' in i):
        aug_mcount=aug_mcount+1
    if ('speech only' in i):
        aug_scount=aug_scount+1
    if ('pointing only' in i):
        aug_pcount=aug_pcount+1
    if('multimodal' not in i and 'speech only' not in i and  'pointing only' not in i):
        aug_nocount=aug_nocount+1
        print(aug_index,': Augdata: ',i)
        print()
    aug_index=aug_index+1
        
for j in Origdata["train"]['output']:
    if ('multimodal' in j):
        orig_mcount=orig_mcount+1
    if ('speech only' in j):
        orig_scount=orig_scount+1
    if ('pointing only' in j):
        orig_pcount=orig_pcount+1
    if('multimodal' not in j and 'speech only' not in j and  'pointing only' not in j):
        orig_nocount=orig_nocount+1
        print(orig_index,': origdata: ',j)
        print()
    orig_index=orig_index+1
    

print('Multimodal count before augmentation:', orig_mcount)
print('Not executed multimodal events before augmentation:', "36")
print('Multimodal count after augmentation:', aug_mcount)
print('Number of aug multimodal REs: ', aug_mcount-orig_mcount)
print('Synthesized multimodal count:', "5632")
print('Total of multimodal REs: ', aug_mcount+5632 )
print()
print('Pointing count before augmentation:', orig_pcount)
print('Pointing count after augmentation:', aug_pcount)
print()
print('Speech count before augmentation:', orig_scount)
print('Not executed speech events before augmentation:', 453-36)
print('Speech count after augmentation:', aug_scount)
print('Number of aug speech REs: ', aug_scount-orig_scount)
print()
print('Not annotated orig_modality: ', orig_nocount)
print('Not annotated aug_modality: ', aug_nocount)
print()
print()
print('Total of original-augmented data: ', aug_scount+aug_mcount+aug_pcount)
print('Total of synthesized augmented data: ', aug_scount+aug_mcount+aug_pcount+5632)




Multimodal count before augmentation: 574
Not executed multimodal events before augmentation: 36
Multimodal count after augmentation: 2296
Number of aug multimodal REs:  1722
Synthesized multimodal count: 5632
Total of multimodal REs:  7928

Pointing count before augmentation: 7681
Pointing count after augmentation: 7681

Speech count before augmentation: 2152
Not executed speech events before augmentation: 417
Speech count after augmentation: 6550
Number of aug speech REs:  4398

Not annotated orig_modality:  0
Not annotated aug_modality:  0


Total of original-augmented data:  16527
Total of synthesized augmented data:  22159
